# U-Lab-MS segmentation pipeline


## Imports


In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
import subprocess
import warnings
from pathlib import Path

import cv2
import numpy as np
import torch
from pytorch_lightning import Trainer, loggers

from augmentations import get_transforms
from datasets import get_train_val_dataloaders
from models import SegmentationModel

warnings.filterwarnings(action="ignore")
from segmentation_models_pytorch.losses import DiceLoss
from sklearn.metrics import f1_score


## Pipeline methods


In [ ]:
def get_trainer(name):
    tb_logger = loggers.TensorBoardLogger(save_dir="logs/", name=name)
    trainer = Trainer(
        auto_select_gpus=True,
        accelerator="auto",
        max_epochs=200,
        logger=tb_logger,
        log_every_n_steps=10,
    )
    return trainer


In [ ]:
def get_trained_model(name, model_type, transform_type, encoder_weights, upscaling):
    transform_train = None
    if transform_type is not None:
        transform_train = get_transforms(transform_type)

    train_dataloader_full, _ = get_train_val_dataloaders(
        split_percent=1.00,
        transform_train=transform_train,
        transform_val=None,
        include_massachusetts=False,
        num_workers=0,
        batch_size=4,
        upscaling=upscaling,
    )
    model = SegmentationModel(model_type, encoder_weights)
    trainer = get_trainer(name)
    trainer.fit(model, train_dataloader_full)
    return model


In [ ]:
def generate_predicted_masks(model, name, upscaling):
    idx = 0
    predictions_dir = Path(f"./data/test/{name}/")
    if upscaling == "vdsr":
        test_images_dir = Path("./data/test/images_800/")
    else:
        test_images_dir = Path("./data/test/images/")

    test_paths = list(test_images_dir.glob("*.png"))
    predictions_dir.mkdir(exist_ok=True)

    for img_path in test_paths:
        print(f"{idx}/{len(test_paths)}")
        idx += 1
        img = cv2.imread(str(img_path))
        img = (img.transpose(2, 0, 1) / 255).astype(np.float32)[None, :]

        pred_mask = model.predict_full_mask(img).cpu().numpy()[0]

        cv2.imwrite(
            str(predictions_dir / img_path.stem) + ".png",
            255 * pred_mask.transpose(1, 2, 0),
        )


In [ ]:
def generate_ensemble_predicted_masks(models, name, upscaling_array):
    idx = 0
    predictions_dir = Path(f"./data/test/{name}/")
    test_images_dir = Path("./data/test/images/")
    test_images_800_dir = Path("./data/test/images_800/")

    test_paths = list(test_images_dir.glob("*.png"))
    test_800_paths = list(test_images_800_dir.glob("*.png"))
    predictions_dir.mkdir(exist_ok=True)

    for img_path, img_800_path in zip(test_paths, test_800_paths):
        print(f"{idx}/{len(test_paths)}")
        idx += 1
        pred_mask = None
        for model, upscaling in zip(models, upscaling_array):
            if upscaling == "vdsr":
                img = cv2.imread(str(img_800_path))
            else:
                img = cv2.imread(str(img_path))
            img = (img.transpose(2, 0, 1) / 255).astype(np.float32)[None, :]

            prediction = model.predict_full_mask(img).cpu().numpy()[0]
            if pred_mask is None:
                pred_mask = prediction
            else:
                pred_mask += prediction

        pred_mask /= len(models)
        cv2.imwrite(
            str(predictions_dir / img_path.stem) + ".png",
            255 * pred_mask.transpose(1, 2, 0),
        )


In [ ]:
def evaluate_ensemble(models, upscaling_array):
    idx = 0
    train_images_dir = Path("./data/training/images/")
    train_images_800_dir = Path("./data/training/images_800/")

    mask_images_dir = Path("./data/training/groundtruth/")

    train_paths = list(train_images_dir.glob("*.png"))
    train_800_paths = list(train_images_800_dir.glob("*.png"))

    mask_paths = list(mask_images_dir.glob("*.png"))

    dice_evals = []
    f1_evals = []

    dice = DiceLoss("binary", from_logits=False)

    for img_path, img_800_path, mask_path in zip(
        train_paths, train_800_paths, mask_paths
    ):
        print(f"{idx}/{len(train_paths)}")
        idx += 1

        pred_mask = None
        for model, upscaling in zip(models, upscaling_array):
            if upscaling == "vdsr":
                img = cv2.imread(str(img_800_path))
            else:
                img = cv2.imread(str(img_path))
            img = (img.transpose(2, 0, 1) / 255).astype(np.float32)[None, :]

            if pred_mask is None:
                pred_mask = model.predict_full_mask(img).cpu().numpy()[0]
            else:
                pred_mask += model.predict_full_mask(img).cpu().numpy()[0]

        pred_mask /= len(models)

        pred_mask = torch.from_numpy(pred_mask.reshape(1, 1, 400, 400))

        mask = cv2.imread(str(mask_path))
        mask = (mask[:, :, :1].transpose(2, 0, 1) / 255).astype(np.float32)
        labels = torch.from_numpy(mask.reshape(1, 400, 400)).int().long()

        pred_mask[pred_mask >= 0.25] = 1
        pred_mask[pred_mask < 0.25] = 0

        dice_eval = dice(pred_mask, labels)
        dice_eval = dice_eval.data.cpu().numpy()
        dice_evals.append(dice_eval)

        f1_eval = f1_score(pred_mask.reshape(-1), labels.reshape(-1))
        print(dice_eval, f1_eval)
        f1_evals.append(f1_eval)

    print(f"Training Dice loss: {sum(dice_evals) / len(dice_evals)}")
    print(f"Training F1 score: {sum(f1_evals) / len(f1_evals)}")


In [ ]:
def generate_submission_file(name):
    submission_path = Path("./kaggle_submissions")
    submission_path.mkdir(exist_ok=True)
    cmd = f'python ./data/mask_to_submission.py --submission_filename="{submission_path / name}.csv" --base_dir="./data/test/{name}/"'
    subprocess.call(cmd)


In [ ]:
def perform_model_pipeline(
    name, model_type, transform_type, encoder_weights, upscaling
):
    model = get_trained_model(
        name, model_type, transform_type, encoder_weights, upscaling
    )
    generate_predicted_masks(model, name, upscaling)
    generate_submission_file(name)


In [ ]:
def perform_trained_model_pipeline(name, checkpoint_dir, model_type, upscaling):
    model = SegmentationModel.load_from_checkpoint(checkpoint_dir, seg_model=model_type)
    generate_predicted_masks(model, name, upscaling)
    generate_submission_file(name)


In [ ]:
def perform_ensemble_pipeline(
    ensemble_name,
    model_names,
    model_types,
    transform_type_array,
    encoder_weights_array,
    upscaling_array,
):
    models = [
        get_trained_model(*params)
        for params in zip(
            model_names,
            model_types,
            transform_type_array,
            encoder_weights_array,
            upscaling_array,
        )
    ]
    generate_ensemble_predicted_masks(models, ensemble_name, upscaling_array)
    generate_submission_file(ensemble_name)
    evaluate_ensemble(models, upscaling_array)


In [ ]:
def perform_trained_ensemble_pipeline(
    ensemble_name, checkpoint_paths, model_types, upscaling_array
):
    models = [
        SegmentationModel.load_from_checkpoint(checkpoint_path, seg_model=model_type)
        for checkpoint_path, model_type in zip(checkpoint_paths, model_types)
    ]
    generate_ensemble_predicted_masks(models, ensemble_name, upscaling_array)
    generate_submission_file(ensemble_name)
    evaluate_ensemble(models, upscaling_array)


In [ ]:
def perform_ensemble_eval_pipeline(checkpoint_paths, model_types, upscaling_array):
    models = [
        SegmentationModel.load_from_checkpoint(checkpoint_path, seg_model=model_type)
        for checkpoint_path, model_type in zip(checkpoint_paths, model_types)
    ]
    return evaluate_ensemble(models, upscaling_array)


## U-Lab-MS


In [ ]:
perform_ensemble_pipeline(
    ensemble_name="U-Lab-MS",
    model_names=[
        "U-lab-MS-U-Net-Small",
        "U-lab-MS-DeepLabV3Plus-Small",
        "U-lab-MS-U-Net-Big",
        "U-lab-MS-DeepLabV3Plus-Big",
    ],
    model_types=["unet", "deeplabv3plus", "unet", "deeplabv3plus"],
    transform_type_array=["rcf", "rcf", "rcf512", "rcf512"],
    encoder_weights_array=["imagenet", "imagenet", "imagenet", "imagenet"],
    upscaling_array=[None, None, "vdsr", "vdsr"],
)


## Models used for ablation studies


### U-Net256 (-A, -P)


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="unet_small_no_augmentations_not_pretrained",
    model_type="unet",
    transform_type="resize_384",
    encoder_weights=None,
    upscaling=None,
)


### U-Net256 (-A)


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="unet_small_no_augmentations_pretrained",
    model_type="unet",
    transform_type="resize_384",
    encoder_weights="imagenet",
    upscaling=None,
)


### U-Net256 (-P)


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="unet_small_with_augmentations_not_pretrained",
    model_type="unet",
    transform_type="rcf",
    encoder_weights=None,
    upscaling=None,
)


### U-Net256


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="unet_small_with_augmentations_pretrained",
    model_type="unet",
    transform_type="rcf",
    encoder_weights="imagenet",
    upscaling=None,
)


### U-Net-ASPP256


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="unet_aspp",
    model_type="unet",
    transform_type="rcf",
    encoder_weights=None,
    upscaling=None,
)


### DeepLabV3Plus256


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="deeplab_small_with_augmentations_pretrained",
    model_type="deeplabv3plus",
    transform_type="rcf",
    encoder_weights="imagenet",
    upscaling=None,
)


### U-Net-512


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="unet_big_with_augmentations_pretrained",
    model_type="unet",
    transform_type="rcf512",
    encoder_weights="imagenet",
    upscaling="vdsr",
)


### DeepLabV3Plus512


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="deeplab_big_with_augmentations_pretrained",
    model_type="deeplabv3plus",
    transform_type="rcf512",
    encoder_weights="imagenet",
    upscaling="vdsr",
)


### U-Net-Edge


In [ ]:
torch.cuda.empty_cache()
perform_model_pipeline(
    name="unet-edge",
    model_type="edgemap_fused_unet",
    transform_type="rcf",
    encoder_weights="imagenet",
    upscaling=None,
)


### U-Lab-256


In [ ]:
perform_ensemble_pipeline(
    ensemble_name="U-Lab-small",
    model_names=["U-lab-small-U-Net-Small", "U-lab-small-DeepLabV3Plus-Small"],
    model_types=["unet", "deeplabv3plus"],
    transform_type_array=["rcf", "rcf"],
    encoder_weights_array=["imagenet", "imagenet"],
    upscaling_array=[None, None],
)


### U-Lab-512


In [ ]:
perform_ensemble_pipeline(
    ensemble_name="U-Lab-big",
    model_names=["U-lab-big-U-Net-Big", "U-lab-big-DeepLabV3Plus-Big"],
    model_types=["unet", "deeplabv3plus"],
    transform_type_array=["rcf512", "rcf512"],
    encoder_weights_array=["imagenet", "imagenet"],
    upscaling_array=["vdsr", "vdsr"],
)
